In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.metrics import  mean_squared_error,  mean_absolute_error, r2_score, make_scorer
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test_df.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test_df.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

In [ ]:
# check for missing values
train_df.isna().mean()

In [ ]:
test_df.isna().mean()

In [ ]:
# correlation between the features and labels
plt.figure(figsize = (15,10))
sns.heatmap(train_df.corr())

No correlation can be found between any of the features.

## Data Preprocessing

In [ ]:
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

In [ ]:
X = train_df.drop('loss', axis=1) 
y = train_df['loss'] # features

In [ ]:
print(X)
print('---------------------------------------')
print(y)

# Splitting data into trains set and test set. Model building, training and testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

R2 score is pretty bad, we need to try other models

## XGBRegressor

In [ ]:
xgbr = XGBRegressor(random_state = 42, learning_rate = 0.25)
xgbr.fit(X_train, y_train)

In [ ]:
ypred = xgbr.predict(X_test)
rmse = mean_squared_error(ypred, y_test, squared=False)
print(print('RMSE:',rmse))

## Light GBM

In [ ]:
lgbmr = LGBMRegressor()
lgbmr.fit(X_train, y_train)
ypred = lgbmr.predict(X_test)
rmse = mean_squared_error(ypred, y_test, squared=False)
print(print('RMSE:',rmse))

## CatBoost Regressor

In [ ]:
cat_boost = CatBoostRegressor('learning_rate': 0.0112, 'iterations': 19562, 
                              'max_bin': 152, 'min_data_in_leaf': 273, 'l2_leaf_reg': 0.91729332782104, 
                              'subsample': 0.6160764186759223, 'leaf_estimation_method': 'Newton','random_seed': 228,
            'loss_function': 'RMSE','eval_metric': 'RMSE, bootstrap_type': 'Bernoulli','task_type': 'GPU')

In [ ]:
cat_boost.fit(X_train, y_train)
y_pred = cat_boost.predict(X_test)
rmse = mean_squared_error(y_pred, y_test, squared=False)
print(print('RMSE:',rmse))

# VOTING REGRESSOR

In [ ]:
reg1 = XGBRegressor()
reg2 = CatBoostRegressor()
reg3 = LGBMRegressor()
ereg = VotingRegressor(estimators=[('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(X_train, y_train)

In [ ]:
y_pred = ereg.predict(X_test)
rmse = mean_squared_error(y_pred, y_test, squared=False)
print(print('RMSE:',rmse))

hyperparameter optimization

# Predicting on test dataset

preprocessing on test dataset

In [ ]:
test_df.head()

In [ ]:
predictions = ereg.predict(test_df)
predictions

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
sample_submission['loss'] = predictions
sample_submission.reset_index()
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)